In [241]:
import json as js
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

In [242]:
main_page='https://realty.yandex.ru'

In [243]:
html_page='C:/Users/dskorokhodov/Documents/realty.html'
with open(html_page,'r',encoding='utf-8') as file:
    r=BeautifulSoup(file,'lxml')

In [244]:
ads=r.find_all('li',{'class':'uLU7v5dzwi8DtxwItem uLU7v5dzwi8DtxwItem_view_desktop uLU7v5dzwi8DtxwItem_format_full uLU7v5dzwi8Dtxw__list-item uLU7v5dzwi8Dtxw__list-item_type_offer'})

In [245]:
# Застройщик
developer=ads[0].find('div',{'class':'uLU7v5dzwi8DtxwAuthor__name'}).text

In [246]:
# Тип жилого фонда
newbuilding=ads[0].find('div',{'class':'uLU7v5dzwi8DtxwItem__building'}).text

In [247]:
# Ближайшая станция метро
metro_location= ads[0].find('span',{'class':'MetroStation__title'}).text

In [248]:
# Время до ближайшей станции метро
metro_distance = ads[0].find('span',{'class':'MetroWithTime__distance'}).text

In [249]:
# Добираться пешком или транспортом до ближайшей станции метро
if ads[0].find('i',{'class':'Icon Icon_type_small-bus'}):
    is_afoot=0
elif ads[0].find('i',{'class':'Icon Icon_type_small-pedestrian'}):
    is_afoot=1

In [250]:
square_n_rooms= ads[0].find('a',{'class':'Link Link_js_inited Link_size_m Link_theme_islands SerpItemLink uLU7v5dzwi8DtxwItem__link'}).text
square,number_of_rooms=square_n_rooms.split(',')

In [251]:
description=ads[0].find('p',{'class':'uLU7v5dzwi8DtxwItem__description'}).text

### Добавляем ссылку на рекламное объявление

In [252]:
ad_link=ads[0].find('a',{'class':'Link Link_js_inited Link_size_m Link_theme_islands SerpItemLink uLU7v5dzwi8DtxwItem__link'}).get('href')

In [253]:
ad_link

'https://realty.yandex.ru/offer/5205561881435530979/?isExact=YES'

In [260]:
dat=[]

In [261]:
for i in range(10):
    print(ad_link)
    dat+=offer_page_scrapping(ad_link)

https://realty.yandex.ru/offer/5205561881435530979/?isExact=YES


AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
df = pd.DataFrame(data=dat)

## Отдельно парсим страницу с объявлением

In [239]:
def offer_page_scrapping(ad_link):
    array=[]
    ### Получаем ответ на наш запрос от страницы
    with requests.get(ad_link) as response_ad_link:
        if response_ad_link:
            response_ad_link.encoding='utf-8'
            soupformat_page=BeautifulSoup(response_ad_link.text,'lxml')
        else:
            print("An error occured in response_ad_link")

    ### Число просмотров объявления
    count_views=soupformat_page.find('span',{'class':'ViewsCount'}).text
    count_views=int(count_views.replace(u'\xa0',u'').split(':')[-1])

    ### Дата размещения объявления
    offer_date=soupformat_page.find('div',{'class':'OfferPublishedDate OfferBaseMetaInfo__item'}).text
    offer_date=offer_date.replace(u'\xa0',u' ')

    ### Описание сделки
    offer_deal_des=soupformat_page.find('span',{'class':'OfferDealDescription__sell-info'}).text
  
    ### Дата сдачи дома,тип сделки
    flat_deadline_date,flat_deal_type=soupformat_page.find('p',{'class':'OfferSellInfo'}).text.split(',')

    offer_tech_des=soupformat_page.find_all('ul',{'class':'ColumnsList OfferTechDescription__list'})
    offer_tech_des=offer_tech_des[0].find_all('div',{'class':'ColumnsList__item'})

    ### Общая площадь квартиры, жилая площадь, площадь кухни, количество комнат, этаж квартиры 
    flat_total_square,flat_houseroom,flat_kitchen_square,flat_number_rooms,flat_floor=[el.text.replace(u'\xa0',u' ').replace(u',',u'.') for el in offer_tech_des]

    building_info=soupformat_page.find('ul',{'class':'ColumnsList BuildingInfo__features-list'})

    building_info=building_info.find_all('span',{'class':'BuildingInfoFeature__by-yandex-data'})

    ### Застройщик, дата сдачи, материал дома, этажность дома, наличие лифта, охрана дома, парковка 
    developer_name,house_date,house_material,house_floors,house_elevator,house_security,house_parking=[el.text.replace(u' (по данным Яндекса)',u'') for el in building_info]

    district_info=soupformat_page.find_all('div',{'class':'CardShortcutsItem__info'})

    descr=[el.find('div',{'class':'CardShortcutsItem__descr'}).text for el in district_info]

    del descr[0:2]
    ### Доступность каршеринга , транспортная доступность, стоимость жилья в районе, стоимость аренды в районе, окупаемость недвижимости в районе
    access_carsharing,access_transport,distinct_flat_price,distinct_flat_lease,distinct_payback=descr

    ### Ссылка на станицу застройщика
    developer_link=soupformat_page.find('a',{'class':'Link Link_js_inited Link_size_m Link_theme_islands CardDevBadge__name'}).get('href')

    developer_link='https://realty.yandex.ru'+developer_link
    
    dev_page_scrap =developer_page_scrapping(developer_link)
    
    return [
            count_views, offer_date, offer_deal_des,
            flat_deadline_date, flat_deal_type, flat_total_square,
            flat_houseroom, flat_kitchen_square, flat_number_rooms,
            flat_floor, developer_name, house_date,
            house_material, house_floors, house_elevator,
            house_security, house_parking, access_carsharing,
            access_transport, distinct_flat_price, distinct_flat_lease, 
            distinct_payback, developer_link, dev_page_scrap,
            ]

## Отдельно парсим страницу застройщика

In [240]:
def developer_page_scrapping(developer_link):
    with requests.get(developer_link) as response_dev_link:
        if response_dev_link:
            response_dev_link.encoding='utf-8'
            soupformat_page_dev=BeautifulSoup(response_dev_link.text,'lxml')
        else:
            print("An error has occured in response_dev_link")

    dev_card=soupformat_page_dev.find_all('span',{'class':'DeveloperCardAbout__object-text'})

    # Построено домов, домов строится, год учреждения организации застройщика
    houses_built,houses_in_building,foundation_date=[el.text.replace(u'\xa0',u' ') for el in dev_card]
    return  [ houses_built, houses_in_building, foundation_date, ] 

In [235]:

parced = js.loads(r)


TypeError: the JSON object must be str, bytes or bytearray, not BeautifulSoup